In [ ]:
!pip install pandas==1.3.2
import numpy as np
import os, sys
import h5py
from optparse import OptionParser 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import svm
from joblib import dump, load
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump, load
import pickle
from sklearn.metrics import precision_score, recall_score, confusion_matrix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/MyDrive', force_remount=True)

Mounted at /MyDrive


In [ ]:
global model_svm

In [ ]:
def createModel(modelname="log"):
    from sklearn.linear_model import SGDClassifier
    global model_svm

    # SVM classifier trained online with stochastic gradient descent
    model_svm = SGDClassifier(loss="hinge", penalty="l2")
    if modelname == "log":
        # Logistic Regresion classifier trained online with stochastic gradient descent
        model_svm = SGDClassifier(loss="log", penalty="l2")  
        print("Using Logistic Regression...")
    if modelname == "non-linear-svm":
        model_svm = svm.SVC(kernel='rbf')
        print("Using RBF kernel SVM...")
    else:
        print("Using Hinge Loss SVM...")
        

def train( X_train, y_train, classes, batchSize=512):
    global model_svm
    best_score=0
    X_count = X_train.shape[0]
    batchCount= X_count // batchSize

    j=0
    shuffledRange = range(X_count)
    shuffledX = X_train[shuffledRange,]
    shuffledY = [y_train[i] for i in shuffledRange]
    for i in range(0, batchCount):  # Iterate over "mini-batches" of 1000 samples each
        j+=1
        y_train_batch = shuffledY[i*batchSize :(i +1)* batchSize]
        X_train_batch = shuffledX[i*batchSize :(i +1)* batchSize,]
        # vectorizer.fit_transform(train_data[i:i + batchSize])
        # Update the classifier with documents in the current mini-batch
        #print("X_train_batch",X_train_batch)
        #print("y_train_batch",y_train_batch)
        model_svm.partial_fit(X_train_batch, y_train_batch, classes=classes)

In [ ]:
dataset_path = "/MyDrive/MyDrive/Lab/Microservices/Bottleneck detection/dataset"
datasets = ["ticket_booking_interence_%d.pkl","media_service_interence_%d.pkl","hotel_reservation_interence_%d.pkl","social_network_interence_%d.pkl"]
datasets = ["training_data_tt_30_nip.pkl","training_data_ms_30_nip.pkl","training_data_hr_30_nip.pkl","training_data_sn_30_nip.pkl"]
datasets = ["social_network_interence_30.pkl"]
non_interference_percentages = [1,10,30,50,70]
non_interference_percentages = [30,10,1]
model_path_svm = os.path.join(dataset_path,"svm_model")

# loop for diff interference %
#for non_interference_percentage in non_interference_percentages:
#  print("\n\n\n NON INTERFERENCE PERCENTAGE %d"%non_interference_percentage)
  
  # file templates for 30% 10% and 1%
  # for file_name_template in datasets:
  #   print(file_name_template)
  #   with open(os.path.join(dataset_path, file_name_template%non_interference_percentage),"rb") as data_f:
  #     dat = pickle.load(data_f)
    # print(type(dat))
    # print(dat.shape)
    #print(dat)

for file_name in datasets:
    print(file_name)
    with open(os.path.join(dataset_path, file_name),"rb") as data_f:
      dat = pickle.load(data_f)
    dat['label'] = dat['label'].astype(int)
    X = dat.drop(columns = ["label"])
    y = dat.loc[:, "label"]
    epochs_svm = 5
    nclasses_svm = len(pd.unique(dat['label']))
    classes = pd.unique(dat['label'])
    classes.sort()
    print(classes)
    batchsize_svm = 512
    non_intereference_index = -1
    interference_start = 0
    interference_end = -1
    # if "_sn_" in file_name:
    #   non_intereference_index = 0
    #   interference_start = 1
    #   interference_end = nclasses_svm

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    createModel(modelname="linear")

    for i in range(epochs_svm):
      train(X_train.to_numpy(), y_train.to_numpy(), classes, batchsize_svm)
      dump(model_svm, model_path_svm)


    y_pred=model_svm.predict(X_test)
    print("accuracy",model_svm.score(X_test, y_test))
    # print("precision_score",precision_score(y_test,y_pred, labels = classes, average="weighted"))
    # print("recall_score",recall_score(y_test,y_pred, labels = classes, average="weighted"))
    # print("precision_score",precision_score(y_test,y_pred, labels = classes, average="macro"))
    # print("recall_score",recall_score(y_test,y_pred, labels = classes, average="macro"))
    # print("precision_score",precision_score(y_test,y_pred, labels = classes, average="micro"))
    # print("recall_score",recall_score(y_test,y_pred, labels = classes, average="micro"))
    per_class_precision = precision_score(y_test,y_pred, labels = classes, average=None)
    per_class_recall = recall_score(y_test,y_pred, labels = classes, average=None)
    print("Precision micro ", precision_score(y_test,y_pred, labels = classes, average="micro"))
    print("Recall micro ", recall_score(y_test,y_pred, labels = classes, average="micro"))
    print("precision_score NI ",per_class_precision[non_intereference_index])
    print("precision_score I ",np.mean(per_class_precision[interference_start:interference_end]))
    print("recall_score NI ",per_class_recall[non_intereference_index])
    print("recall_score I ",np.mean(per_class_recall[interference_start:interference_end]))
    print(confusion_matrix(y_test, y_pred))

social_network_interence_30.pkl
[-1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
Using Hinge Loss SVM...


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


accuracy 0.7352568514194421
Precision micro  0.7352568514194421
Recall micro  0.7352568514194421
precision_score NI  0.7401703398015629
precision_score I  0.7370554048172694
recall_score NI  0.8921556176445686
recall_score I  0.7715097870590745
[[29603  4233  4294 ...  3544  4759  4217]
 [  957 41829   162 ...   142   133   194]
 [ 1032   136 42246 ...   106   148   138]
 ...
 [ 1043   150   142 ... 42394   158   138]
 [  789  1042  1038 ...   945 20799  1067]
 [ 1073   128   112 ...   130   130 42149]]


In [ ]:
# just for one run

per_class_precision = precision_score(y_test,y_pred, labels = classes, average=None)
per_class_recall = recall_score(y_test,y_pred, labels = classes, average=None)
print(per_class_precision)
print(per_class_recall)
print("Precision micro ", precision_score(y_test,y_pred, labels = classes, average="micro"))
print("Recall micro ", recall_score(y_test,y_pred, labels = classes, average="micro"))
print("Precision macro ", precision_score(y_test,y_pred, labels = classes, average="macro"))
print("Recall macro ", recall_score(y_test,y_pred, labels = classes, average="macro"))
print("precision_score NI ",per_class_precision[non_intereference_index])
print("precision_score I ",np.mean(per_class_precision[interference_start:interference_end]))
print("recall_score NI ",per_class_recall[non_intereference_index])
print("recall_score I ",np.mean(per_class_recall[interference_start:interference_end]))
print(confusion_matrix(y_test, y_pred))

[0.49410803 0.74991932 0.74754481 0.74491511 0.73550001 0.75763926
 0.82056318 0.70635119 0.75277451 0.73380546 0.74951312 0.7671143
 0.73428017 0.72997028 0.75413356 0.65289    0.75348155 0.75671261
 0.84707379 0.74701469 0.75999137 0.73021343 0.73874472 0.73592334
 0.89339824 0.75461001 0.7497559  0.53552142 0.74784251 0.77197902
 0.69543266 0.74017034]
[0.18701986 0.88459586 0.89190559 0.89289161 0.89141053 0.89116191
 0.77925643 0.89252681 0.88608392 0.89148496 0.89257627 0.88972484
 0.89238972 0.89451504 0.89238979 0.40035422 0.89358269 0.88954764
 0.42256024 0.89129384 0.89453414 0.41381265 0.89615287 0.89735978
 0.38333653 0.8913466  0.89414181 0.37760196 0.89323356 0.890911
 0.43710071 0.89215562]
Precision micro  0.7352568514194421
Recall micro  0.7352568514194421
Precision micro  0.7371527465355286
Recall micro  0.7752799692648711
precision_score NI  0.7401703398015629
precision_score I  0.7370554048172694
recall_score NI  0.8921556176445686
recall_score I  0.7715097870590745

In [ ]:
confusion_matrix_arr = confusion_matrix(y_test, y_pred)

FP = confusion_matrix_arr.sum(axis=0) - np.diag(confusion_matrix_arr)  
FN = confusion_matrix_arr.sum(axis=1) - np.diag(confusion_matrix_arr)
TP = np.diag(confusion_matrix_arr)
TN = confusion_matrix_arr.sum() - (FP + FN + TP)
print(FP, FN, TP, TN)
print(FP.sum())
print(FN.sum())

[30309 13949 14267 14573 15227 13531  8131 17611 13856 15336 14148 12781
 15323 15625 13740 10095 13772 13555  3606 14364 13350  7281 14960 15294
  2146 13720 14096 15482 14288 12522  9109 14796] [128685   5457   5120   5105   5158   5166  10533   5101   5424   5146
   5095   5218   5106   4981   5082  28440   5013   5235  27295   5173
   4984  27916   4902   4875  28932   5143   5000  29422   5065   5191
  26785   5095] [29603 41829 42246 42557 42342 42299 37183 42362 42190 42276 42334 42100
 42343 42239 42144 18988 42094 42161 19974 42414 42273 19707 42302 42621
 17985 42191 42233 17850 42375 42394 20799 42149] [1438803 1566165 1565767 1565165 1564673 1566404 1571553 1562326 1565930
 1564642 1565823 1567301 1564628 1564555 1566434 1569877 1566521 1566449
 1576525 1565449 1566793 1572496 1565236 1564610 1578337 1566346 1566071
 1564646 1565672 1567293 1570707 1565360]
430843
430843


In [ ]:
for file_name in datasets:
    print(file_name)
    with open(os.path.join(dataset_path, file_name),"rb") as data_f:
      dat = pickle.load(data_f)
    dat['label'] = dat['label'].astype(int)
    print(dat.groupby('label').count())

social_network_interence_30.pkl
       1_nginx_compose-post  2_mediaservice_upload  3_userservice_upload  \
label                                                                      
-1                   790000                 790000                790000   
 1                   237000                 237000                237000   
 2                   237000                 237000                237000   
 3                   237000                 237000                237000   
 4                   237000                 237000                237000   
 5                   237000                 237000                237000   
 6                   237000                 237000                237000   
 7                   237000                 237000                237000   
 8                   237000                 237000                237000   
 9                   237000                 237000                237000   
 10                  237000                 237000      